# **Air Quality**

---
---

# Data Pre-processing

In [1]:
import pandas as pd
#pip install pandas openpyxl (para leer excels)
import numpy as np
import folium
import zipfile

In [2]:
pd.set_option('display.max_columns', None)

#### **Upload and view the polluting data for Spain in 2020**
- Concatenate the files to be able to work better with them.
- Process the data before the EDA.

In [3]:
C6H6_HH_22 = pd.read_csv('../data/raw/time data 2022/C6H6_HH_2022.csv', sep = ';')
CO_HH_22 = pd.read_csv('../data/raw/time data 2022/CO_HH_2022.csv', sep= ';')
NO2_HH_22 = pd.read_csv('../data/raw/time data 2022/NO2_HH_2022.csv', sep = ';')
NOx_HH_22 = pd.read_csv('../data/raw/time data 2022/NOx_HH_2022.csv', sep = ';')
O3_HH_22 = pd.read_csv('../data/raw/time data 2022/O3_HH_2022.csv', sep = ';')
PM10_HH_22 = pd.read_csv('../data/raw/time data 2022/PM10_HH_2022.csv', sep = ';')
PM25_HH_22 = pd.read_csv('../data/raw/time data 2022/PM25_HH_2022.csv', sep = ';')
SO2_HH_22 = pd.read_csv('../data/raw/time data 2022/SO2_HH_2022.csv', sep = ';')

In [4]:
pollutants = pd.concat([C6H6_HH_22, CO_HH_22, NO2_HH_22, NOx_HH_22, O3_HH_22, PM10_HH_22, PM25_HH_22, SO2_HH_22])
print(pollutants.shape)

(819614, 32)


In [5]:
pollutants.head()

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANNO,MES,DIA,H01,H02,H03,H04,H05,H06,H07,H08,H09,H10,H11,H12,H13,H14,H15,H16,H17,H18,H19,H20,H21,H22,H23,H24
0,1,59,8,30,01059008_30_59,2022,1,1,1.58,0.73,0.55,0.68,0.65,0.72,0.56,0.59,0.56,0.84,0.87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,59,8,30,01059008_30_59,2022,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.24,0.23,0.22,0.23,0.58,0.24,0.54,0.69,0.32,0.17,0.17
2,1,59,8,30,01059008_30_59,2022,1,4,0.16,0.16,0.18,0.18,0.14,0.15,0.22,0.18,0.16,0.18,0.18,0.13,0.17,0.11,0.17,0.18,0.19,0.25,0.25,0.28,0.25,0.20,0.20,0.22
3,1,59,8,30,01059008_30_59,2022,1,5,0.23,0.26,0.22,0.20,0.21,0.23,0.28,0.31,0.34,0.40,0.35,0.29,0.27,0.30,0.48,0.33,0.45,0.58,0.57,0.51,0.51,0.51,0.48,0.41
4,1,59,8,30,01059008_30_59,2022,1,6,0.36,0.31,0.32,0.33,0.32,0.33,0.33,0.37,0.54,0.45,0.37,0.32,0.30,0.28,0.30,0.26,0.29,0.36,0.43,0.32,0.32,0.30,0.30,0.28


In [6]:
pollutants.to_csv('../data/interim/pollutants.csv', index= False)
pollutants = pd.read_csv('../data/interim/pollutants.csv')

In [7]:
total_nan = pollutants.isna().sum().sum()
total_nan

367595

In [8]:
pollutants = pollutants.dropna()
pollutants.shape

(719282, 32)

In [9]:
pollutants.drop(['PUNTO_MUESTREO'], axis=1, inplace= True)

In [10]:
pollutants.shape

(719282, 31)

In [11]:
# Crear una columna 'fecha' con el año, mes y día
pollutants['FECHA'] = pd.to_datetime(pollutants[['ANNO', 'MES', 'DIA']].astype(str).agg('-'.join, axis= 1), errors='coerce', format='%Y-%m-%d')
pollutants.drop(['ANNO', 'MES', 'DIA'], axis= 1, inplace= True)

In [12]:
pollutants.head()

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,H01,H02,H03,H04,H05,H06,H07,H08,H09,H10,H11,H12,H13,H14,H15,H16,H17,H18,H19,H20,H21,H22,H23,H24,FECHA
2,1,59,8,30,0.16,0.16,0.18,0.18,0.14,0.15,0.22,0.18,0.16,0.18,0.18,0.13,0.17,0.11,0.17,0.18,0.19,0.25,0.25,0.28,0.25,0.20,0.20,0.22,2022-01-04
3,1,59,8,30,0.23,0.26,0.22,0.20,0.21,0.23,0.28,0.31,0.34,0.40,0.35,0.29,0.27,0.30,0.48,0.33,0.45,0.58,0.57,0.51,0.51,0.51,0.48,0.41,2022-01-05
4,1,59,8,30,0.36,0.31,0.32,0.33,0.32,0.33,0.33,0.37,0.54,0.45,0.37,0.32,0.30,0.28,0.30,0.26,0.29,0.36,0.43,0.32,0.32,0.30,0.30,0.28,2022-01-06
5,1,59,8,30,0.25,0.24,0.25,0.26,0.32,0.32,0.28,0.25,0.30,0.27,0.30,0.39,0.28,0.23,0.28,0.29,0.28,0.32,0.31,0.23,0.25,0.29,0.29,0.28,2022-01-07
6,1,59,8,30,0.25,0.25,0.23,0.23,0.32,0.29,0.23,0.28,0.55,0.34,0.36,0.28,0.27,0.27,0.25,0.20,0.23,0.25,0.25,0.23,0.20,0.22,0.22,0.19,2022-01-08


In [13]:
measured_excel = pd.read_excel('../data/raw/Metainformacion2022.xlsx', sheet_name= 'Magnitudes')
measured_excel.head()

,MAGNITUD,NOMBRE,ABREVIATURA,UNIDAD,GRUPO
0,1,DIÓXIDO DE AZUFRE,SO2,µg/m3,Contaminantes con valor legislado
1,6,MONÓXIDO DE CARBONO,CO,mg/m3,Contaminantes con valor legislado
2,7,MONÓXIDO DE NITRÓGENO,NO,µg/m3,NaN
3,8,DIÓXIDO DE NITRÓGENO,NO2,µg/m3,Contaminantes con valor legislado
4,9,"PARTÍCULAS EN SUSPENSIÓN <2,5µM",PM2.5,µg/m3,Contaminantes con valor legislado


In [14]:
# Leer una hoja específica del archivo Excel
stations_excel= pd.read_excel('../data/raw/Metainformacion2022.xlsx', sheet_name='Estaciones')
stations_excel.head(3)

,COD_LOCAL,PROVINCIA,MUNICIPIO,ESTACION,COD_ESTACION_DEM,N_RED,NOMBRE,FECHA_INI,FECHA_FIN,LATITUD_G,LONGITUD_G,ALTITUD,N_CCAA,N_PROVINCIA,N_MUNICIPIO,TIPO_ESTACION,TIPO_AREA,TIPO_SUBAREA_RURAL,EST,ZONA,DIRECCION
0,1022001,1,22,1,ES1672A,CCAA País Vasco,EL CIEGO,2004-05-10,NaT,42.51833,-2.61944,480,PAÍS VASCO,ÁLAVA,ELCIEGO,TRAFICO,SUBURBANA,NaN,ST,RESIDENCIAL,"C/GABRIEL CELAYA 8, ELCIEGO"
1,1036004,1,36,4,ES1349A,CCAA País Vasco,LLODIO,1994-01-01,NaT,43.14407,-2.96337,122,PAÍS VASCO,ÁLAVA,LLODIO,TRAFICO,SUBURBANA,NaN,ST,RESIDENCIAL/COMERCIAL,"C/ LAMUZA, S/N"
2,1051001,1,51,1,ES1544A,CCAA País Vasco,AGURAIN,1998-01-01,NaT,42.84900,-2.39370,594,PAÍS VASCO,ÁLAVA,SALVATIERRA O AGURAIN,FONDO,SUBURBANA,NaN,SF,RESIDENCIAL/INDUSTRIAL,CUARTEL DE LA ERTZANTZA


In [15]:
columnas_a_modificar = ['LATITUD_G', 'LONGITUD_G']

for columna in columnas_a_modificar:
    # Asegurarse de que todos los valores no-NaN sean strings y aplicar la operación de reemplazo
    stations_excel[columna] = stations_excel[columna].apply(lambda x: str(x).replace(',', '.') if pd.notnull(x) else x)

# Después de asegurarse de que todos los valores son strings y de reemplazar las comas
# Convertimos toda la columna a float
for columna in columnas_a_modificar:
    stations_excel[columna] = stations_excel[columna].astype(float)

stations_excel.head()

,COD_LOCAL,PROVINCIA,MUNICIPIO,ESTACION,COD_ESTACION_DEM,N_RED,NOMBRE,FECHA_INI,FECHA_FIN,LATITUD_G,LONGITUD_G,ALTITUD,N_CCAA,N_PROVINCIA,N_MUNICIPIO,TIPO_ESTACION,TIPO_AREA,TIPO_SUBAREA_RURAL,EST,ZONA,DIRECCION
0,1022001,1,22,1,ES1672A,CCAA País Vasco,EL CIEGO,2004-05-10,NaT,42.51833,-2.61944,480,PAÍS VASCO,ÁLAVA,ELCIEGO,TRAFICO,SUBURBANA,NaN,ST,RESIDENCIAL,"C/GABRIEL CELAYA 8, ELCIEGO"
1,1036004,1,36,4,ES1349A,CCAA País Vasco,LLODIO,1994-01-01,NaT,43.14407,-2.96337,122,PAÍS VASCO,ÁLAVA,LLODIO,TRAFICO,SUBURBANA,NaN,ST,RESIDENCIAL/COMERCIAL,"C/ LAMUZA, S/N"
2,1051001,1,51,1,ES1544A,CCAA País Vasco,AGURAIN,1998-01-01,NaT,42.84900,-2.39370,594,PAÍS VASCO,ÁLAVA,SALVATIERRA O AGURAIN,FONDO,SUBURBANA,NaN,SF,RESIDENCIAL/INDUSTRIAL,CUARTEL DE LA ERTZANTZA
3,1055001,1,55,1,ES1489A,CCAA País Vasco,VALDEREJO,1998-01-01,NaT,42.87520,-3.23170,911,PAÍS VASCO,ÁLAVA,VALDEGOVÍA,FONDO,RURAL,REMOTA,RFREM,NATURALEZA,PARQUE NATURAL DE VALDEREJO
4,1059008,1,59,8,ES1502A,CCAA País Vasco,AVENIDA GASTEIZ,1998-01-01,NaT,42.85480,-2.68070,517,PAÍS VASCO,ÁLAVA,VITORIA-GASTEIZ,TRAFICO,URBANA,NaN,UT,RESIDENCIAL/COMERCIAL,"AVDA. GASTEIZ, 93"


In [16]:
# Crear un mapa base
m = folium.Map(location=[stations_excel['LATITUD_G'].mean(), stations_excel['LONGITUD_G'].mean()], zoom_start=6)

# Añadir puntos al mapa
for idx, row in stations_excel.iterrows():
    folium.Marker(location=[row['LATITUD_G'], row['LONGITUD_G']],
                  popup=row['N_MUNICIPIO'] if 'NombrePunto' in stations_excel.columns else "Sin Nombre").add_to(m)

# Guardar el mapa en un archivo HTML
m.save('mapa.html')
#Aqui no se ve nada, creo que hay qeu hacerlo en un navegador normal

In [17]:
measured_dict = {}

for index, row in measured_excel.iterrows():
    measured_dict[row['MAGNITUD']] = row['ABREVIATURA']

measured_dict

{1: 'SO2',
 6: 'CO',
 7: 'NO',
 8: 'NO2',
 9: 'PM2.5',
 10: 'PM10',
 12: 'NOX',
 14: 'O3',
 17: 'As',
 19: 'Pb',
 27: 'BAP',
 28: 'Cd',
 30: 'C6H6',
 62: 'NI'}

In [18]:
# Crear diccionario de las magnitudes 
magnitud_dict = {
    1: 'SO2',
    6: 'CO',
    7: 'NO',
    8: 'NO2',
    9: 'PM2.5',
    10: 'PM10',
    12: 'NOX',
    14: 'O3',
    17: 'As',
    19: 'Pb',
    27: 'BAP',
    28: 'Cd',
    30: 'C6H6',
    62: 'NI'
}

In [19]:
# Ejemplo: df = pd.DataFrame({'Clave1': [...], 'Clave2': [...], 'Clave3': [...], 'Valor': [...]})

# Crear el diccionario
dicc_latitude = {(row['PROVINCIA'], row['MUNICIPIO'], row['ESTACION']): row['LATITUD_G'] for index, row in stations_excel.iterrows()}

In [20]:
# Ejemplo: df = pd.DataFrame({'Clave1': [...], 'Clave2': [...], 'Clave3': [...], 'Valor': [...]})

# Crear el diccionario
dicc_longitude = {(row['PROVINCIA'], row['MUNICIPIO'], row['ESTACION']): row['LONGITUD_G'] for index, row in stations_excel.iterrows()}

In [21]:
# Función para mapear cada fila al valor del diccionario
def mapear_latitud(row):
    # Crea una tupla con los valores de la fila
    clave = (row['PROVINCIA'], row['MUNICIPIO'], row['ESTACION'])
    # Devuelve el valor correspondiente del diccionario
    return dicc_latitude.get(clave, None)  # Retorna None si la clave no existe

# Aplica la función a cada fila y crea una nueva columna
pollutants['LATITUD'] = pollutants.apply(mapear_latitud, axis=1)

In [22]:
# Función para mapear cada fila al valor del diccionario
def mapear_longitud(row):
    # Crea una tupla con los valores de la fila
    clave = (row['PROVINCIA'], row['MUNICIPIO'], row['ESTACION'])
    # Devuelve el valor correspondiente del diccionario
    return dicc_longitude.get(clave, None)  # Retorna None si la clave no existe

# Aplica la función a cada fila y crea una nueva columna
pollutants['LONGITUD'] = pollutants.apply(mapear_longitud, axis=1)

In [23]:
pollutants.head(30)

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,H01,H02,H03,H04,H05,H06,H07,H08,H09,H10,H11,H12,H13,H14,H15,H16,H17,H18,H19,H20,H21,H22,H23,H24,FECHA,LATITUD,LONGITUD
2,1,59,8,30,0.16,0.16,0.18,0.18,0.14,0.15,0.22,0.18,0.16,0.18,0.18,0.13,0.17,0.11,0.17,0.18,0.19,0.25,0.25,0.28,0.25,0.20,0.20,0.22,2022-01-04,42.8548,-2.6807
3,1,59,8,30,0.23,0.26,0.22,0.20,0.21,0.23,0.28,0.31,0.34,0.40,0.35,0.29,0.27,0.30,0.48,0.33,0.45,0.58,0.57,0.51,0.51,0.51,0.48,0.41,2022-01-05,42.8548,-2.6807
4,1,59,8,30,0.36,0.31,0.32,0.33,0.32,0.33,0.33,0.37,0.54,0.45,0.37,0.32,0.30,0.28,0.30,0.26,0.29,0.36,0.43,0.32,0.32,0.30,0.30,0.28,2022-01-06,42.8548,-2.6807
5,1,59,8,30,0.25,0.24,0.25,0.26,0.32,0.32,0.28,0.25,0.30,0.27,0.30,0.39,0.28,0.23,0.28,0.29,0.28,0.32,0.31,0.23,0.25,0.29,0.29,0.28,2022-01-07,42.8548,-2.6807
6,1,59,8,30,0.25,0.25,0.23,0.23,0.32,0.29,0.23,0.28,0.55,0.34,0.36,0.28,0.27,0.27,0.25,0.20,0.23,0.25,0.25,0.23,0.20,0.22,0.22,0.19,2022-01-08,42.8548,-2.6807
7,1,59,8,30,0.18,0.19,0.20,0.19,0.21,0.24,0.27,0.28,0.26,0.29,0.31,0.31,0.29,0.25,0.29,0.27,0.23,0.22,0.20,0.17,0.17,0.11,0.09,0.10,2022-01-09,42.8548,-2.6807
8,1,59,8,30,0.07,0.06,0.08,0.14,0.15,0.16,0.13,0.19,0.23,0.24,0.38,0.24,0.18,0.14,0.20,0.14,0.21,0.25,0.41,0.30,0.43,0.47,0.24,0.18,2022-01-10,42.8548,-2.6807
9,1,59,8,30,0.17,0.14,0.15,0.11,0.14,0.20,0.23,0.46,0.42,0.26,0.68,0.21,0.22,0.20,0.20,0.46,0.68,0.35,0.37,0.36,0.41,0.48,0.36,0.70,2022-01-11,42.8548,-2.6807
11,1,59,8,30,0.50,0.52,0.32,0.92,0.55,0.67,0.71,0.58,0.60,0.70,0.67,0.46,0.43,0.39,0.42,0.39,0.47,0.58,0.78,0.84,0.80,1.00,1.39,1.86,2022-01-13,42.8548,-2.6807
12,1,59,8,30,2.00,1.16,0.85,0.67,0.67,1.18,0.78,0.88,2.02,1.12,0.64,0.50,0.52,0.60,0.59,0.42,0.48,0.52,0.78,1.18,1.28,2.35,2.20,2.12,2022-01-14,42.8548,-2.6807


In [24]:
# Dos claves son 'CLAVE1' y 'CLAVE2' y el valor es 'VALOR'
diccionario_municipios = {(row['PROVINCIA'], row['MUNICIPIO']): row['N_MUNICIPIO'] for index, row in stations_excel.iterrows()}
diccionario_municipios

{(1, 22): 'ELCIEGO',
 (1, 36): 'LLODIO',
 (1, 51): 'SALVATIERRA O AGURAIN',
 (1, 55): 'VALDEGOVÍA',
 (1, 59): 'VITORIA-GASTEIZ',
 (2, 3): 'ALBACETE',
 (3, 2): 'AGOST',
 (3, 9): 'ALCOY/ALCOI',
 (3, 14): 'ALICANTE/ALACANT',
 (3, 31): 'BENIDORM',
 (3, 65): 'ELCHE/ELX',
 (3, 66): 'ELDA',
 (3, 99): 'ORIHUELA',
 (3, 105): 'PINOSO',
 (3, 133): 'TORREVIEJA',
 (4, 13): 'ALMERÍA',
 (4, 22): 'BÉDAR',
 (4, 24): 'BENAHADUX',
 (4, 32): 'CARBONERAS',
 (4, 35): 'CUEVAS DEL ALMANZORA',
 (4, 66): 'NÍJAR',
 (4, 902): 'EJIDO (EL)',
 (5, 19): 'ÁVILA',
 (6, 15): 'BADAJOZ',
 (6, 16): 'BARCARROTA',
 (6, 22): 'BURGUILLOS DEL CERRO',
 (6, 70): 'JEREZ DE LOS CABALLEROS',
 (6, 81): 'MEDINA DE LAS TORRES',
 (6, 83): 'MÉRIDA',
 (6, 158): 'ZAFRA',
 (7, 3): 'ALCÚDIA',
 (7, 10): 'BUNYOLA',
 (7, 15): 'CIUTADELLA DE MENORCA',
 (7, 19): 'ESCORCA',
 (7, 26): 'EIVISSA',
 (7, 32): 'MAHÓN',
 (7, 40): 'PALMA DE MALLORCA',
 (7, 42): 'POLLENÇA',
 (7, 44): 'POBLA (SA)',
 (7, 46): 'SANT ANTONI DE PORTMANY',
 (7, 54): 'SANTA EULAL

In [25]:
# Crear un diccionario vacío para los nombres de las provincias
diccionario_provincias = {}

for index, row in stations_excel.iterrows():
    diccionario_provincias[row['PROVINCIA']] = row['N_PROVINCIA']

diccionario_provincias

{1: 'ÁLAVA',
 2: 'ALBACETE',
 3: 'ALICANTE/ALACANT',
 4: 'ALMERÍA',
 5: 'ÁVILA',
 6: 'BADAJOZ',
 7: 'BALEARS (ILLES)',
 8: 'BARCELONA',
 9: 'BURGOS',
 10: 'CÁCERES',
 11: 'CÁDIZ',
 12: 'CASTELLÓN/CASTELLÓ',
 13: 'CIUDAD REAL',
 14: 'CÓRDOBA',
 15: 'CORUÑA (A)',
 16: 'CUENCA',
 17: 'GIRONA',
 18: 'GRANADA',
 19: 'GUADALAJARA',
 20: 'GUIPÚZCOA',
 21: 'HUELVA',
 22: 'HUESCA',
 23: 'JAÉN',
 24: 'LEÓN',
 25: 'LLEIDA',
 26: 'RIOJA (LA)',
 27: 'LUGO',
 28: 'MADRID',
 29: 'MÁLAGA',
 30: 'MURCIA',
 31: 'NAVARRA',
 32: 'OURENSE',
 33: 'ASTURIAS',
 34: 'PALENCIA',
 35: 'PALMAS (LAS)',
 36: 'PONTEVEDRA',
 37: 'SALAMANCA',
 38: 'SANTA CRUZ DE TENERIFE',
 39: 'CANTABRIA',
 40: 'SEGOVIA',
 41: 'SEVILLA',
 42: 'SORIA',
 43: 'TARRAGONA',
 44: 'TERUEL',
 45: 'TOLEDO',
 46: 'VALENCIA',
 47: 'VALLADOLID',
 48: 'VIZCAYA',
 49: 'ZAMORA',
 50: 'ZARAGOZA',
 51: 'CEUTA'}

In [26]:
dict_ccaa = {(row['PROVINCIA'], row['MUNICIPIO']): row['N_CCAA'] 
                    for index, row in stations_excel.iterrows()}

dict_ccaa

{(1, 22): 'PAÍS VASCO',
 (1, 36): 'PAÍS VASCO',
 (1, 51): 'PAÍS VASCO',
 (1, 55): 'PAÍS VASCO',
 (1, 59): 'PAÍS VASCO',
 (2, 3): 'CASTILLA-LA MANCHA',
 (3, 2): 'COMUNIDAD VALENCIANA',
 (3, 9): 'COMUNIDAD VALENCIANA',
 (3, 14): 'COMUNIDAD VALENCIANA',
 (3, 31): 'COMUNIDAD VALENCIANA',
 (3, 65): 'COMUNIDAD VALENCIANA',
 (3, 66): 'COMUNIDAD VALENCIANA',
 (3, 99): 'COMUNIDAD VALENCIANA',
 (3, 105): 'COMUNIDAD VALENCIANA',
 (3, 133): 'COMUNIDAD VALENCIANA',
 (4, 13): 'ANDALUCÍA',
 (4, 22): 'ANDALUCÍA',
 (4, 24): 'ANDALUCÍA',
 (4, 32): 'ANDALUCÍA',
 (4, 35): 'ANDALUCÍA',
 (4, 66): 'ANDALUCÍA',
 (4, 902): 'ANDALUCÍA',
 (5, 19): 'CASTILLA Y LEÓN',
 (6, 15): 'EXTREMADURA',
 (6, 16): 'EXTREMADURA',
 (6, 22): 'EXTREMADURA',
 (6, 70): 'EXTREMADURA',
 (6, 81): 'EXTREMADURA',
 (6, 83): 'EXTREMADURA',
 (6, 158): 'EXTREMADURA',
 (7, 3): 'BALEARES (ISLAS)',
 (7, 10): 'BALEARES (ISLAS)',
 (7, 15): 'BALEARES (ISLAS)',
 (7, 19): 'BALEARES (ISLAS)',
 (7, 26): 'BALEARES (ISLAS)',
 (7, 32): 'BALEARES (ISLAS)

In [27]:
# Crear clave para agregar la CCAA
pollutants['CLAVE_COMBINADA'] = pollutants.apply(lambda x: (x['PROVINCIA'], x['MUNICIPIO']), axis=1)
pollutants['N_CCAA'] = pollutants['CLAVE_COMBINADA'].map(dict_ccaa)
pollutants.drop('CLAVE_COMBINADA', axis=1, inplace=True)

In [28]:
# Crear la clave combinada en el DataFrame objetivo.
pollutants['LOC_MUNIC'] = pollutants.apply(lambda x: (x['PROVINCIA'], x['MUNICIPIO']), axis=1)

# Usar el método .map() para mapear los nombres de municipio usando el diccionario.
pollutants['N_MUNICIPIO'] = pollutants['LOC_MUNIC'].map(diccionario_municipios)

pollutants.drop('LOC_MUNIC', axis=1, inplace=True)

In [29]:
pollutants['MAGNITUD'] = pollutants['MAGNITUD'].map(measured_dict)
pollutants['PROVINCIA'] = pollutants['PROVINCIA'].map(diccionario_provincias)
pollutants

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,H01,H02,H03,H04,H05,H06,H07,H08,H09,H10,H11,H12,H13,H14,H15,H16,H17,H18,H19,H20,H21,H22,H23,H24,FECHA,LATITUD,LONGITUD,N_CCAA,N_MUNICIPIO
2,ÁLAVA,59,8,C6H6,0.16,0.16,0.18,0.18,0.14,0.15,0.22,0.18,0.16,0.18,0.18,0.13,0.17,0.11,0.17,0.18,0.19,0.25,0.25,0.28,0.25,0.20,0.20,0.22,2022-01-04,42.8548,-2.6807,PAÍS VASCO,VITORIA-GASTEIZ
3,ÁLAVA,59,8,C6H6,0.23,0.26,0.22,0.20,0.21,0.23,0.28,0.31,0.34,0.40,0.35,0.29,0.27,0.30,0.48,0.33,0.45,0.58,0.57,0.51,0.51,0.51,0.48,0.41,2022-01-05,42.8548,-2.6807,PAÍS VASCO,VITORIA-GASTEIZ
4,ÁLAVA,59,8,C6H6,0.36,0.31,0.32,0.33,0.32,0.33,0.33,0.37,0.54,0.45,0.37,0.32,0.30,0.28,0.30,0.26,0.29,0.36,0.43,0.32,0.32,0.30,0.30,0.28,2022-01-06,42.8548,-2.6807,PAÍS VASCO,VITORIA-GASTEIZ
5,ÁLAVA,59,8,C6H6,0.25,0.24,0.25,0.26,0.32,0.32,0.28,0.25,0.30,0.27,0.30,0.39,0.28,0.23,0.28,0.29,0.28,0.32,0.31,0.23,0.25,0.29,0.29,0.28,2022-01-07,42.8548,-2.6807,PAÍS VASCO,VITORIA-GASTEIZ
6,ÁLAVA,59,8,C6H6,0.25,0.25,0.23,0.23,0.32,0.29,0.23,0.28,0.55,0.34,0.36,0.28,0.27,0.27,0.25,0.20,0.23,0.25,0.25,0.23,0.20,0.22,0.22,0.19,2022-01-08,42.8548,-2.6807,PAÍS VASCO,VITORIA-GASTEIZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
819608,CEUTA,1,1,SO2,1.53,1.33,1.58,1.41,1.43,2.88,1.76,1.51,1.42,1.44,1.55,1.41,1.65,1.88,2.22,2.43,3.32,6.62,19.11,17.15,8.29,29.35,24.51,24.16,2022-12-13,35.8930,-5.3147,CEUTA,CEUTA
819609,CEUTA,1,1,SO2,1.64,1.50,1.47,1.44,1.58,1.38,1.12,1.22,1.46,1.18,1.37,2.75,8.84,15.18,15.03,25.54,12.58,38.78,17.84,8.01,11.66,3.28,5.70,5.62,2022-12-16,35.8930,-5.3147,CEUTA,CEUTA
819610,CEUTA,1,1,SO2,2.30,2.15,2.17,2.05,1.82,1.74,1.85,2.48,2.71,3.53,3.34,2.98,2.99,2.44,2.72,2.27,2.07,2.00,1.79,1.65,1.68,1.39,1.30,1.28,2022-12-25,35.8930,-5.3147,CEUTA,CEUTA
819611,CEUTA,1,1,SO2,1.40,1.22,1.22,1.11,1.04,1.16,1.15,1.23,1.18,1.43,1.65,1.91,2.09,2.99,3.31,3.77,2.82,2.47,2.79,2.77,2.66,2.51,2.42,2.52,2022-12-26,35.8930,-5.3147,CEUTA,CEUTA


In [30]:
# Nombres de las columnas de horas en orden
columnas_horas = ['H01', 'H02', 'H03', 'H04', 'H05', 'H06', 'H07', 'H08', 'H09', 'H10', 
                  'H11', 'H12', 'H13', 'H14', 'H15', 'H16', 'H17', 'H18', 'H19', 'H20', 
                  'H21', 'H22', 'H23', 'H24']

# Nuevo orden de las columnas
nuevo_orden = ['FECHA', 'N_CCAA', 'PROVINCIA', 'N_MUNICIPIO', 'ESTACION', 'MAGNITUD', 'LATITUD', 'LONGITUD'] + columnas_horas

pollutants = pollutants[nuevo_orden]
pollutants

,FECHA,N_CCAA,PROVINCIA,N_MUNICIPIO,ESTACION,MAGNITUD,LATITUD,LONGITUD,H01,H02,H03,H04,H05,H06,H07,H08,H09,H10,H11,H12,H13,H14,H15,H16,H17,H18,H19,H20,H21,H22,H23,H24
2,2022-01-04,PAÍS VASCO,ÁLAVA,VITORIA-GASTEIZ,8,C6H6,42.8548,-2.6807,0.16,0.16,0.18,0.18,0.14,0.15,0.22,0.18,0.16,0.18,0.18,0.13,0.17,0.11,0.17,0.18,0.19,0.25,0.25,0.28,0.25,0.20,0.20,0.22
3,2022-01-05,PAÍS VASCO,ÁLAVA,VITORIA-GASTEIZ,8,C6H6,42.8548,-2.6807,0.23,0.26,0.22,0.20,0.21,0.23,0.28,0.31,0.34,0.40,0.35,0.29,0.27,0.30,0.48,0.33,0.45,0.58,0.57,0.51,0.51,0.51,0.48,0.41
4,2022-01-06,PAÍS VASCO,ÁLAVA,VITORIA-GASTEIZ,8,C6H6,42.8548,-2.6807,0.36,0.31,0.32,0.33,0.32,0.33,0.33,0.37,0.54,0.45,0.37,0.32,0.30,0.28,0.30,0.26,0.29,0.36,0.43,0.32,0.32,0.30,0.30,0.28
5,2022-01-07,PAÍS VASCO,ÁLAVA,VITORIA-GASTEIZ,8,C6H6,42.8548,-2.6807,0.25,0.24,0.25,0.26,0.32,0.32,0.28,0.25,0.30,0.27,0.30,0.39,0.28,0.23,0.28,0.29,0.28,0.32,0.31,0.23,0.25,0.29,0.29,0.28
6,2022-01-08,PAÍS VASCO,ÁLAVA,VITORIA-GASTEIZ,8,C6H6,42.8548,-2.6807,0.25,0.25,0.23,0.23,0.32,0.29,0.23,0.28,0.55,0.34,0.36,0.28,0.27,0.27,0.25,0.20,0.23,0.25,0.25,0.23,0.20,0.22,0.22,0.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
819608,2022-12-13,CEUTA,CEUTA,CEUTA,1,SO2,35.8930,-5.3147,1.53,1.33,1.58,1.41,1.43,2.88,1.76,1.51,1.42,1.44,1.55,1.41,1.65,1.88,2.22,2.43,3.32,6.62,19.11,17.15,8.29,29.35,24.51,24.16
819609,2022-12-16,CEUTA,CEUTA,CEUTA,1,SO2,35.8930,-5.3147,1.64,1.50,1.47,1.44,1.58,1.38,1.12,1.22,1.46,1.18,1.37,2.75,8.84,15.18,15.03,25.54,12.58,38.78,17.84,8.01,11.66,3.28,5.70,5.62
819610,2022-12-25,CEUTA,CEUTA,CEUTA,1,SO2,35.8930,-5.3147,2.30,2.15,2.17,2.05,1.82,1.74,1.85,2.48,2.71,3.53,3.34,2.98,2.99,2.44,2.72,2.27,2.07,2.00,1.79,1.65,1.68,1.39,1.30,1.28
819611,2022-12-26,CEUTA,CEUTA,CEUTA,1,SO2,35.8930,-5.3147,1.40,1.22,1.22,1.11,1.04,1.16,1.15,1.23,1.18,1.43,1.65,1.91,2.09,2.99,3.31,3.77,2.82,2.47,2.79,2.77,2.66,2.51,2.42,2.52


In [31]:
clean_pollutants = pollutants.to_csv('../data/processed/clean_pollutants.csv', index= False)